# Import dependencies

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Check if GPU is available

In [2]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# Select GPU as device if available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


# Mount the Drive unit

# Definicion del Torch Dataset personalizado

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import Compose
import os
import cv2

class GenshinDataSet(Dataset):
    def __init__(self, directory: str, transforms: Compose =None ) -> None:
        self.directory = directory
        self.characters = os.listdir(directory)
        self.transforms = transforms
        self.images = []
        self.labels = []

        # Load image paths and labels from your file structure (modify accordingly)
        for character in self.characters:
            category_path = os.path.join(directory, character)
            label = character  # Assuming category folder name represents the label

            for image_file in os.listdir(category_path):
                image_path = os.path.join(category_path, image_file)
                self.images.append(image_path)
                self.labels.append(label)


    def __getitem__(self, index) -> tuple[any, str]:
        image = cv2.imread(self.images[index])
        label = self.labels[index]

        if image.shape[-1] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms:
            image = self.transforms(image)

        return image, label

    def __len__(self) -> int:
        return len(self.images)

# Crear Data set

In [5]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

In [6]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)
print(type(train_dataset))
# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
<class 'torchvision.datasets.cifar.CIFAR10'>
Files already downloaded and verified


# ConvNet

In [ ]:
class ConvNet(nn.Module):
  def __init__(self,category_count):
    super(ConvNet,self).__init__()
    self.convolution_layer1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3) # Convolution layer 1: 3 input channels (RGB), 32 output channels (32 filters), 3x3 kernel size
    self.convolution_layer2 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3) # Convolution layer 2: 32 input channels (32 kernels from previous layer), 32 output channels (32 filters), 3x3 kernel size
    self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2) # Max pooling layer: 2x2 kernel size, stride 2 (reduces image size by 2)

    self.convolution_layer3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3) # Convolution layer 3: 32 input channels (32 kernels from previous layer), 64 output channels (64 filters), 3x3 kernel size
    self.convolution_layer4 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
    self.max_pool2 = nn.MaxPool2d(kernel_size=2,stride=2) # Max pooling layer: 2x2 kernel size, stride 2 (reduces image size by 2)

    self.fully_connected1 = nn.Linear(1600,128) # Fully connected layer 1: 64*5*5 input features (64 filters, 5x5 image size), 128 output features
    self.relu = nn.ReLU() # ReLU activation function
    self.fully_connected2 = nn.Linear(128,category_count) # Output layer: linear layer

  def forward(self,x):
    output = self.convolution_layer1(x)
    output = self.convolution_layer2(output)
    output = self.max_pool(output)

    output = self.convolution_layer3(output)
    output = self.convolution_layer4(output)
    output = self.max_pool2(output)

    output = output.reshape(output.size(0),-1)

    output = self.fully_connected1(output)
    output = self.relu(output)
    output = self.fully_connected2(output)
    return output

# Configurando hiperparametro

In [ ]:
model = ConvNet(num_classes) # Create the model with the number of classes required

lossFunction = nn.CrossEntropyLoss() # Set loss function as CrossEntropyLoss

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9) # Set thr optimizer as a Stochastic Gradient Descent with the learning rate, weight decay and momentum

total_step = len(train_loader) # Set the total step as the length of the train loader

# Entrenamiento

In [ ]:
if torch.cuda.is_available():
      model.cuda()

for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = lossFunction(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Prueba

In [ ]:
with torch.no_grad():
    if torch.cuda.is_available():
      model.cuda()
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))



In [ ]:
# prompt: save the model as a file

torch.save(model.state_dict(), 'model.pt')


# Open from file

In [ ]:
# prompt: open and make inference from  model.pt

# Import the necessary libraries
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Load the model
model = ConvNet(num_classes)
model.load_state_dict(torch.load('model.pt'))

# Define the data transform
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the data
data_dir = 'path/to/data'
dataset = ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Make inference
with torch.no_grad():
    for images, labels in dataloader:
        # Move the images to the device
        images = images.to(device)

        # Get the model's predictions
        outputs = model(images)

        # Get the predicted class labels
        _, predicted = torch.max(outputs, 1)

        # Print the predicted class labels
        print(predicted)
